- 總共有一千筆訓練資料、40個 features，二元分類問題，練習 features scaling、建模、調參數等步驟
- 每天最多上傳 10 次結果
- 請在 private / public leaderboard 上取得 0.7 以上的準確率
- 可多參考別人的 Kernel，學習別人的寫法與思路，完成自己的 Kaggle 競賽

- This is a binary classification task, You are evaluated on classification accuracy (the percentage of labels you predict correctly).  The training set has 1000 samples and the testing set has 9000.  Your prediction should be a 9000 x 1 vector of ones or zeros. You also need an Id column (1 to 9000) and should include a header. The format looks like this:

Id,Solution
1,0
2,1
3,1
...
9000,0

- 這是一個二元分類問題
    - 因此要挑選分類問題的模型，如Logsitic Model、隨機森林、梯度提升機等等。

- 比賽以正確率評斷，因此模型檢測也用此評價。

In [1]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
# read and observe data
url = './day48_kaggle/'
data_train_x = pd.read_csv(url + 'train.csv')
data_train_y = pd.read_csv(url + 'trainLabels.csv')
test = pd.read_csv(url + 'test.csv')
# data_train.describe()
# print(data_train_x.head())
# print(data_train_y.head())
# print(data_train_y.info())
# print(data_train_y.shape)

In [4]:
# logistic

# read
url = './day48_kaggle/'
data_train_x = pd.read_csv(url + 'train.csv')
data_train_y = pd.read_csv(url + 'trainLabels.csv')
data_test_x = pd.read_csv(url + 'test.csv')

# split
x_train, x_test, y_train, y_test = train_test_split(data_train_x, data_train_y, test_size=0.2, random_state=18)

# model
log = LogisticRegression().fit(x_train, y_train)

# predict
y_pred = log.predict(x_test)
log_pred = log.predict_proba(test)[:, 1]

# 評估
print(f'score: {metrics.accuracy_score(y_pred, y_test)}')

score: 0.8


C:\Users\aband\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\aband\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [5]:
# RandomForestClassifier

# read
url = './day48_kaggle/'
data_train_x = pd.read_csv(url + 'train.csv')
data_train_y = pd.read_csv(url + 'trainLabels.csv')
data_test_x = pd.read_csv(url + 'test.csv')

# split
x_train, x_test, y_train, y_test = train_test_split(data_train_x, data_train_y, test_size=0.2, random_state=18)

# model
rfc = RandomForestClassifier().fit(x_train, y_train)

# predict
y_pred = rfc.predict(x_test)
rfc_pred = rfc.predict_proba(test)[:, 1]

# 評估
print(f'score: {metrics.accuracy_score(y_pred, y_test)}')

score: 0.79


C:\Users\aband\AppData\Roaming\Python\Python37\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\aband\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]


In [6]:
# GradientBoostingClassifier

# read
url = './day48_kaggle/'
data_train_x = pd.read_csv(url + 'train.csv')
data_train_y = pd.read_csv(url + 'trainLabels.csv')
data_test_x = pd.read_csv(url + 'test.csv')

# split
x_train, x_test, y_train, y_test = train_test_split(data_train_x, data_train_y, test_size=0.2, random_state=18)

# model
gdbt = GradientBoostingClassifier().fit(x_train, y_train)

# predict
y_pred = gdbt.predict(x_test)
gdbt_pred = gdbt.predict_proba(test)[:, 1]

# 評估
print(f'score: {metrics.accuracy_score(y_pred, y_test)}')

C:\Users\aband\AppData\Roaming\Python\Python37\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


score: 0.865


In [8]:
# 集成

# 滿奇怪的，csv-test資料會少吃一筆，可能是因為沒有column names的關係
# 我自己補了第一筆
blending_pred = log_pred*0.3 + gdbt_pred*0.3 + rfc_pred*0.4
sub = pd.DataFrame({'Id': [i for i in range(1, 9001)], 'Solution': blending_pred})
sub['Solution'] = sub['Solution'].map(lambda x: 1 if x>0.5 else 0)
sub.to_csv('scikit.csv', index=False)             